This script aims to use the hard-coded cluster average method for computing cell type expression signatures, to compare against the negative binomial regression model method.

**Author:** Yiqing Wang

**Date:** 2024-8-3

INPUT: 
1) filtered single cell data (AnnData)
2) (optional) CSV of model-estimated cell type signatures

OUTPUT: 
1) hard-coded average expression of every gene in every cell type
2) (optional) scatter plot showing correlation between average expression of every gene in every cell type and model-estimated expression of every gene in every cell type

1. Loading packages

In [1]:
import os
import scanpy as sc
import cell2location.cluster_averages

Additional packages used in validation (optional):

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

2. Loading filtered single cell data (AnnData object)

In [3]:
dir = "path/to/data/test_results/run_name"
os.chdir(dir)
results_folder = "./hard_code_average"

In [4]:
# Load filtered Anndata, produced by sc_preprocessing.ipynb
adata_ref = sc.read_h5ad(f"{dir}/adata_ref_filtered.h5ad")

In [ ]:
# Inspect cell metadata columns
print(adata_ref.obs.columns)

# Inspect if a raw attribute or any additional layers exist
# If not, we can assume that adata_ref.X contains (filtered) raw data.
print(adata_ref.raw is None)
print(len(adata_ref.layers))

The "label" column contains cell type information (this can be confirmed by checking its content).

There is no raw slot;
there are no layers either.

3. Using hard-coded average method to compute average expression of every gene in every cell type

In [5]:

# Use Cell2location's hard-coded average function to compute cell type averages of genes
aver = cell2location.cluster_averages.cluster_averages.compute_cluster_averages(
    adata=adata_ref, labels="label", use_raw=False, layer=None
    )

In [23]:
# Save results to CSV
aver.to_csv(f"{results_folder}/hard_code_aver.csv")

4. (Optional) comparing average expression of every gene in every cell type with the model-estimated expression of every gene in every cell type.

This section is for validation purposes only. It makes a scatter plot to compare cell type signatures computed by hard-coded averages and NB regression model, similar to the second plot made by Cell2location's plot_QC() method.

In [6]:
# Read model-estimated cell type expression signatures
inf_aver = pd.read_csv(f"{dir}/inf_aver.csv", index_col=0)

# Melt the data columns of the data frames (to a single column) for plotting
aver_melted = aver.reset_index().melt(id_vars=['index'], var_name='cell_type', value_name='value')
inf_aver_melted = inf_aver.reset_index().melt(id_vars=['index'], var_name='cell_type', value_name='value')

In [ ]:
# Make a scatter plot of the hard-coded average vs. the NB model, for all cell types

# Calculate the Pearson correlation coefficient
# Since average expressions in aver_melted include zeros, a small constant is added to avoid log(0).
correlation_coefficient = np.corrcoef(np.log10(aver_melted["value"] + 0.01), 
                                      np.log10(inf_aver_melted["value"]))[0, 1]

# Create a scatter plot
plt.scatter(np.log10(aver_melted["value"] + 0.01), np.log10(inf_aver_melted["value"]), 
            color = "blue", alpha = 0.5)

# Add the correlation coefficient as text on the plot
plt.text(0.05, 0.95, f'Pearson r = {correlation_coefficient:.2f}', 
         transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', color='red')

# Add labels and title
plt.xlabel('hard-coded average')
plt.ylabel('NB model')
plt.title('Hard-coded Average vs. NB Model Cell Type Signatures')

# Save the plot
plt.savefig(f"{results_folder}/scatter_hard-code_average_vs_NB_signatures.png", dpi=300)

# Show the plot
plt.show()

# Close the current figure
plt.close()

The correlation is very high, indicating high similarity between the results of the two methods, which further indicates the lack of batch effect in the single cell reference data. 

Next, I wanted to check the correlation between the hard-coded average of every gene and the model-estimated expression of every gene, for a specific cell type only.

In [ ]:
# Make a scatter plot of the hard-coded average vs. the NB model, for a specific cell type

ct = "Oligo"

# Calculate the Pearson correlation coefficient
correlation_coefficient_ct = np.corrcoef(np.log10(aver[ct] + 0.01), 
                                         np.log10(inf_aver[ct]))[0, 1]

# Create a scatter plot
plt.scatter(np.log10(aver[ct] + 0.01), np.log10(inf_aver[ct]), color = "blue", alpha = 0.5)

# Add the correlation coefficient as text on the plot
plt.text(0.05, 0.95, f'Pearson r = {correlation_coefficient_ct:.2f}', 
         transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', color='red')

# Add labels and title
plt.xlabel('hard-coded average')
plt.ylabel('NB model')
plt.title(f"Hard-coded Average vs. NB Model Cell Type Signatures: {ct}")

# Save the plot
plt.savefig(f"{results_folder}/scatter_{ct}_hard-code_average_vs_NB_signatures.png", dpi=300)

# Show the plot
plt.show()

# Close the current figure
plt.close()

For oligodendrocytes, the similarity between hard-coded average and model-estimated expression levels is even more prominent.